# Appendix: Import needed modules 

In [251]:
## Spell checker from Peter Norvig. Not perfect, but works.
import re, collections

def words(text): return re.findall('[a-z]+', text.lower()) 

def train(features):
    model = collections.defaultdict(lambda: 1)
    for f in features:
        model[f] += 1
    return model

NWORDS = train(words(open('C:/Users/kennd/Downloads/big.txt').read()))

alphabet = 'abcdefghijklmnopqrstuvwxyz'

def edits1(word):
   splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
   deletes    = [a + b[1:] for a, b in splits if b]
   transposes = [a + b[1] + b[0] + b[2:] for a, b in splits if len(b)>1]
   replaces   = [a + c + b[1:] for a, b in splits for c in alphabet if b]
   inserts    = [a + c + b     for a, b in splits for c in alphabet]
   return set(deletes + transposes + replaces + inserts)

def known_edits2(word):
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1) if e2 in NWORDS)

def known(words): return set(w for w in words if w in NWORDS)

def correct(word):
    candidates = known([word]) or known(edits1(word)) or known_edits2(word) or [word]
    return max(candidates, key=NWORDS.get)

In [300]:
## Import all modules needed
## lem is lemmatizing the words
def lem(x):
    sentence = ''
    for x in re.sub('[^a-z0-9 ]','',x).split():
        sentence = sentence + ' ' + lemmatizer.lemmatize(correct(x))
    return sentence

import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from nltk.corpus import stopwords

stops = set(stopwords.words('english'))

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=1,stop_words=stops)

import pandas as pd
df = pd.read_csv('C:/Users/kennd/Desktop/yelp_collate_test.csv')

# Download data and munge munge

In [301]:
df2 = df[['sentence','category']]

In [294]:
# df = df.drop(['wait','ambience','food','service','value','none'],axis=1)

In [295]:
# df2 = df.fillna(0)

In [304]:
df2['s_lower'] = df2['sentence'].apply(lambda x: x.lower())

C:\Users\kennd\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [306]:
## lemmatizing and spell checking the words
## this  pre-processing of text is needed before feeding it into tokenizing by word
df2['s_spell_lem'] = df2['s_lower'].apply(lem)

C:\Users\kennd\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [307]:
df3 = df2[['s_spell_lem','category']]

In [308]:
corp = list(df3.s_spell_lem)

In [309]:
df3.head(10)

,s_spell_lem,category
0,rocky ha been in business since the 1980s and...,others
1,their food is good,food
2,not haute quinine pretension,food
3,everyday well prepared and taste bud pleasing...,food
4,give mostly gone during supper hour and and a...,food
5,he ordered the large version of fish sandwich,food
6,rocky is famous for their fish,food
7,three large dreaded and fried fish file but a...,value
8,needle to say he had no complaint and brought...,value
9,i ordered the meatal drill,food


# Feature Generation

In [310]:
## Gets the count of each word in each sentence
feature = vectorizer.fit_transform(corp)

In [311]:
## Transform into word counts into pandas data frame
featuredf = pd.DataFrame(feature.A, columns=vectorizer.get_feature_names())

In [312]:
## Crude normalization of features
featuredf2 = featuredf.loc[:,featuredf.sum()>= 3]
featuredf2 = featuredf2.loc[:,featuredf2.sum()<= 20]

In [313]:
df5 = pd.concat((df3,featuredf2),axis=1)

In [314]:
X = df5.iloc[:,2:]
Y = df5.iloc[:,1]

In [315]:
df5.head()

,s_spell_lem,category,ago,almost,also,although,always,american,another,area,...,well,went,whole,wing,without,working,worth,would,year,yes
0,rocky ha been in business since the 1980s and...,others,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,their food is good,food,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,not haute quinine pretension,food,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,everyday well prepared and taste bud pleasing...,food,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,give mostly gone during supper hour and and a...,food,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Building the model and in-training-set accuracy

In [323]:
from sklearn.cross_validation import train_test_split 

In [324]:
##Split into train and test at 75/25
train, test = train_test_split(df5,test_size = 0.25)

In [325]:
##Split X & Y
X_train = train.iloc[:,2:]
Y_train = train.iloc[:,1]
X_test = test.iloc[:,2:]
Y_test = test.iloc[:,1]

In [326]:
## Just used NB because that's what the scikit-learn tutorial shows
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train,Y_train)

In [327]:
## tried SVC because its supposedly very good at classifying text
from sklearn.svm import SVC
svc = SVC(gamma=1).fit(X_train,Y_train)

In [328]:
## NB In-training-set accuracy
Y_PRED = clf.predict(X_test)
np.mean(Y_PRED == np.array(Y_test))

0.51578947368421058

In [329]:
## SVC In-training-set accuracy
Y_PRED = svc.predict(X_test)
np.mean(Y_PRED == np.array(Y_test))

0.45263157894736844

# Sentiment Scoring

In [330]:
## Modified off-the-shelf sentiment analyzer
import nltk.sentiment.util
def demo_liu_hu_lexicon2(sentence, plot=False):
    """
    Basic example of sentiment classification using Liu and Hu opinion lexicon.
    This function simply counts the number of positive, negative and neutral words
    in the sentence and classifies it depending on which polarity is more represented.
    Words that do not appear in the lexicon are considered as neutral.

    :param sentence: a sentence whose polarity has to be classified.
    :param plot: if True, plot a visual representation of the sentence polarity.
    """
    from nltk.corpus import opinion_lexicon
    from nltk.tokenize import treebank

    tokenizer = treebank.TreebankWordTokenizer()
    pos_words = 0
    neg_words = 0
    tokenized_sent = [word.lower() for word in tokenizer.tokenize(sentence)]

    x = list(range(len(tokenized_sent))) # x axis for the plot
    y = []

    for word in tokenized_sent:
        if word in opinion_lexicon.positive():
            pos_words += 1
            y.append(1) # positive
        elif word in opinion_lexicon.negative():
            neg_words += 1
            y.append(-1) # negative
        else:
            y.append(0) # neutral

    return (pos_words - neg_words)
#     if pos_words > neg_words:
#         return 'Positive'
#     elif pos_words < neg_words:
#         return 'Negative'
#     elif pos_words == neg_words:
#         return 'Neutral'

    if plot == True:
        _show_plot(x, y, x_labels=tokenized_sent, y_labels=['Negative', 'Neutral', 'Positive'])

In [331]:
df5['sentiment'] = df5['s_spell_lem'].apply(lambda x: demo_liu_hu_lexicon2(x))

In [332]:
## Positive values mean positive sentiment, zero for neutral, negative for..negative
df5.head(20)

,s_spell_lem,category,ago,almost,also,although,always,american,another,area,...,went,whole,wing,without,working,worth,would,year,yes,sentiment
0,rocky ha been in business since the 1980s and...,others,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,their food is good,food,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,not haute quinine pretension,food,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,everyday well prepared and taste bud pleasing...,food,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
4,give mostly gone during supper hour and and a...,food,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,he ordered the large version of fish sandwich,food,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,rocky is famous for their fish,food,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,three large dreaded and fried fish file but a...,value,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1
8,needle to say he had no complaint and brought...,value,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1
9,i ordered the meatal drill,food,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [333]:
df5['business_id'] = df['Business ID']
df5['star'] = df['Review stars']

# Processing Data for Visualization

In [334]:
##Sentiments should be normalized, but whatever for this example
df5.groupby(['business_id','category']).sum().loc[:,['sentiment']]

sentiment
business_id            category           
5UmKMjUEUNdYWqANhGckJw ambience          1
                       food              3
                       others            2
                       service           3
                       value             1
KayYbHCt-RkbGcPdGOThNg ambience          5
                       food              8
                       others            3
                       service           7
                       value             2
UsFtqoBl7naz8AVUBZMjQQ ambience          4
                       fodd              0
                       food             27
                       service           1
                       value             1
WETE_LykpcnrC1sFcQ5EGg food              8
                       other             1
                       service           2
                       value             2
b9WZJp5L1RZr4F1nxclOoQ ambiance          5
                       ambience          4
                       food             46
                       other             1
                       others            3
                       service          15
                       value             5
                       wait              4
fNGIbpazjTRdXgwRY_NIXA ambience          3
                       food             15
                       others            1
                       service           1
                       value             7
mVHrayjG3uZ_RLHkLj-AMg ambience          6
                       food             26
                       others           15
                       service          20
                       value             6
                       wait              1
zaXDakTd3RXyOa7sMrUE1g ambiance          4
                       food              0
                       other             0
                       service           1
                       value             6
                       wait             -1

In [335]:
##Interesting to see how 5 stars generally have positive sentiments on all aspects
df5.groupby(['star','category']).sum().loc[:,['sentiment']]

sentiment
star category           
1.0  food              8
     value             3
3.0  ambience          3
     fodd              0
     food             13
     others            1
     service           4
     value             7
     wait              0
4.0  ambience          7
     food             29
     others           12
     service          10
     value             0
     wait              4
5.0  ambience         13
     food             61
     others           11
     service          24
     value             9
     wait              2